In [156]:
from lowe.acs.ACSClient import ACSClient
from lowe.locations.lookup import name2fips, fips2name, search
import pandas as pd

In [2]:
client = ACSClient()
await client.initialize()

In [68]:
CATHEDRAL_CITY = "0612048"
COACHELLA = "0614260"
DESERT_HS = "0618996"
INDIAN_WELLS = "0636434"
INDIO = "0636448"
LA_QUINTA = "0640354"
PALM_DESERT = "0655184"
PALM_SPRINGS = "0655254"
RANCHO_MIRAGE = "0659500"

cities = [CATHEDRAL_CITY, COACHELLA, DESERT_HS, INDIAN_WELLS, INDIO, LA_QUINTA, PALM_DESERT, PALM_SPRINGS, RANCHO_MIRAGE]

locs = [{"city": cityname} for cityname in cities]

In [167]:
resp = await client.get_acs(
    vars=["S0101"],
    start_year="2012",
    end_year="2019",
    location=locs,
    estimate="5",
    debug=False)

In [168]:
cols_mask = resp.columns.str.contains("AGE AND SEX Estimate Percent|city|location_key")
resp = resp.loc[:, cols_mask]
no_gender_mask = ~resp.columns.str.contains("male|female|Male|Female|MALE|FEMALE")
resp = resp.loc[:, no_gender_mask]
other_mask = ~resp.columns.str.contains("SELECTED AGE CATEGORIES|SUMMARY INDICATORS|PERCENT ALLOCATED")
resp = resp.loc[:, other_mask]

In [169]:
for col in resp.columns:
    print(col)

AGE AND SEX Estimate Percent Total population
AGE AND SEX Estimate Percent Total population AGE 10 to 14 years
AGE AND SEX Estimate Percent Total population AGE 15 to 19 years
AGE AND SEX Estimate Percent Total population AGE 20 to 24 years
AGE AND SEX Estimate Percent Total population AGE 25 to 29 years
AGE AND SEX Estimate Percent Total population AGE 30 to 34 years
AGE AND SEX Estimate Percent Total population AGE 35 to 39 years
AGE AND SEX Estimate Percent Total population AGE 40 to 44 years
AGE AND SEX Estimate Percent Total population AGE 45 to 49 years
AGE AND SEX Estimate Percent Total population AGE 5 to 9 years
AGE AND SEX Estimate Percent Total population AGE 50 to 54 years
AGE AND SEX Estimate Percent Total population AGE 55 to 59 years
AGE AND SEX Estimate Percent Total population AGE 60 to 64 years
AGE AND SEX Estimate Percent Total population AGE 65 to 69 years
AGE AND SEX Estimate Percent Total population AGE 70 to 74 years
AGE AND SEX Estimate Percent Total population 

In [170]:
newcols = []
for col in resp.columns:
    newcols.append(col.replace("AGE AND SEX Estimate Percent Total population AGE ", ""))
resp.columns = newcols

In [171]:
resp["0-14 years perc"] = resp["Under 5 years"].astype(float) + resp["5 to 9 years"].astype(float) + resp["10 to 14 years"].astype(float)
resp["15-24 years perc"] = resp["15 to 19 years"].astype(float) + resp["20 to 24 years"].astype(float)
resp["25-34 years perc"] = resp["25 to 29 years"].astype(float) + resp["30 to 34 years"].astype(float)
resp["35-44 years perc"] = resp["35 to 39 years"].astype(float) + resp["40 to 44 years"].astype(float)
resp["45-64 years perc"] = resp["45 to 49 years"].astype(float) + resp["50 to 54 years"].astype(float) + resp["55 to 59 years"].astype(float) + resp["60 to 64 years"].astype(float)
resp["65+ years perc"] = resp["65 to 69 years"].astype(float) + resp["70 to 74 years"].astype(float) + resp["75 to 79 years"].astype(float) + resp["80 to 84 years"].astype(float) + resp["85 years and over"].astype(float)


resp = resp.rename(columns = {"AGE AND SEX Estimate Percent Total population": "Total population"})

resp["0-14 years"] = resp["0-14 years perc"].astype(float) * resp["Total population"].astype(float) * 0.01
resp["15-24 years"] = resp["15-24 years perc"].astype(float) * resp["Total population"].astype(float) * 0.01
resp["25-34 years"] = resp["25-34 years perc"].astype(float) * resp["Total population"].astype(float) * 0.01
resp["35-44 years"] = resp["35-44 years perc"].astype(float) * resp["Total population"].astype(float) * 0.01
resp["45-64 years"] = resp["45-64 years perc"].astype(float) * resp["Total population"].astype(float) * 0.01
resp["65+ years"] = resp["65+ years perc"].astype(float) * resp["Total population"].astype(float) * 0.01


In [172]:
cols = ["location_key", "0-14 years perc", "15-24 years perc", "25-34 years perc",
             "35-44 years perc","45-64 years perc","65+ years perc",
             "0-14 years", "15-24 years", "25-34 years", 
             "35-44 years","45-64 years","65+ years", "Total population"]

resp = resp[cols]

resp["Total population"] = resp["Total population"].astype(float)

In [173]:
cv = resp.groupby(["year"]).sum()
cv["0-14 years perc"] = cv["0-14 years"] / cv["Total population"]
cv["15-24 years perc"] = cv["15-24 years"] / cv["Total population"]
cv["25-34 years perc"] = cv["25-34 years"] / cv["Total population"]
cv["35-44 years perc"] = cv["35-44 years"] / cv["Total population"]
cv["45-64 years perc"] = cv["45-64 years"] / cv["Total population"]
cv["65+ years perc"] = cv["65+ years"] / cv["Total population"]
cv["location_key"] = "Coachella Valley"

cv = cv[cols]

In [174]:
cv

together = pd.concat([resp, cv])
together

,location_key,0-14 years perc,15-24 years perc,25-34 years perc,35-44 years perc,45-64 years perc,65+ years perc,0-14 years,15-24 years,25-34 years,35-44 years,45-64 years,65+ years,Total population
year,,,,,,,,,,,,,,
2012,cathedral city ca,20.7,16.6,11.5,12.6,24.7,13.7,5312.034,4259.892,2951.13,3233.412,6338.514,3515.694,25662.0


In [138]:
resp["Total population"]

year
2012    25662
        ...  
Name: Total population, Length: 72, dtype: object